# Optimize dataset column types to save memory

Below operations that can be used to optimize the type of columns in the dataset in order to save memory. Ideas were taken from this [post](https://www.dataquest.io/blog/pandas-big-data/).

## Load libraries

In [1]:
# Load libraries
import pandas as pd

## Define global functions

In [2]:
# Function that computes the memory usage
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return '{:03.2f} MB'.format(usage_mb)

## Load data

In [3]:
file_name = 'ours_2019-01-01_to_2020-02-22_coronavirus(es-en)_tweets.csv'

In [4]:
tweets = pd.read_csv('data/'+file_name, parse_dates=['date','formatted_date'])

## Memory saving operations

In [5]:
# Check memory information about the dataset
tweets.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4740401 entries, 0 to 4740400
Data columns (total 22 columns):
 #   Column             Dtype              
---  ------             -----              
 0   tweet_id           int64              
 1   user_id            int64              
 2   tweet              object             
 3   key                object             
 4   key_group          object             
 5   date               datetime64[ns]     
 6   formatted_date     datetime64[ns, UTC]
 7   retweets           int64              
 8   favorites          int64              
 9   mentions           object             
 10  hashtags           object             
 11  geo                float64            
 12  urls               object             
 13  permalink          object             
 14  username           object             
 15  key_mention        object             
 16  key_group_mention  object             
 17  lang               object             
 18  tw

In [6]:
# Check how much memory is occupied by each data type
for dtype in ['float','int','object']:
    selected_dtype = tweets.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print('Average memory usage for {} columns: {:03.2f} MB'.format(dtype,mean_usage_mb))

Average memory usage for float columns: 18.08 MB
Average memory usage for int columns: 32.15 MB
Average memory usage for object columns: 438.98 MB


In [7]:
# Optimize int columns by downcasting them
tweets_int = tweets.select_dtypes(include=['int'])
converted_int = tweets_int.apply(pd.to_numeric, downcast='unsigned')
print('Memory usage of original int columns:')
print(mem_usage(tweets_int))
print('Memory usage of converted int columns:')
print(mem_usage(converted_int))
tweets[converted_int.columns] = converted_int

Memory usage of original int columns:
289.33 MB
Memory usage of converted int columns:
131.10 MB


In [8]:
# Optimize float columns by downcasting them
tweets_float = tweets.select_dtypes(include=['float'])
converted_float = tweets_float.apply(pd.to_numeric, downcast='float')
print('Memory usage of original float columns: {}'.format(mem_usage(tweets_float)))
print('Memory usage of converted float columns: {}'.format(mem_usage(converted_float)))
tweets[converted_float.columns] = converted_float

Memory usage of original float columns: 36.17 MB
Memory usage of converted float columns: 18.08 MB


In [9]:
# Check how much memory we have saved after previous operations
print('Total memory after downcasting int and float columns: {}'.format(mem_usage(tweets)))

Total memory after downcasting int and float columns: 5489.33 MB


In [10]:
# Optimize object types using categoricals
tweets_obj = tweets.select_dtypes(include=['object']).copy()
converted_obj = pd.DataFrame()
threshold_unique_values = 0.2
converted_cols = 0
for col in tweets_obj.columns:
    num_unique_values = len(tweets_obj[col].unique())
    num_total_values = len(tweets_obj[col])
    if num_unique_values / num_total_values < threshold_unique_values:
        converted_obj.loc[:,col] = tweets_obj[col].astype('category')
        converted_cols += 1
    else:
        converted_obj.loc[:,col] = tweets_obj[col]
print('Memory usage of original object columns: {}'.format(mem_usage(tweets_obj)))
print('Memory usage of converted object columns: {}'.format(mem_usage(converted_obj)))
tweets[converted_obj.columns] = converted_obj
print('{} object columns were converted in the process'.format(converted_cols))

Memory usage of original object columns: 5858.06 MB
Memory usage of converted object columns: 3541.84 MB
7 object columns were converted in the process


In [11]:
# Check how much memory we have saved after previous operations
print('Total memory after converting some object columns to categorical columns: {}'.format(mem_usage(tweets)))

Total memory after converting some object columns to categorical columns: 3763.36 MB


In [ ]:
# in case we want to optimize datetime types, no needed here
# date = optimized_tweets.date
# print(mem_usage(date))
# optimized_tweets['date'] = pd.to_datetime(date,format='%Y-%m-%d %H:%M:%S')
# print(mem_usage(optimized_tweets))

In [12]:
# Show final types
dtypes = tweets.drop(['date', 'formatted_date'], axis=1).dtypes
dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]
column_types = dict(zip(dtypes_col, dtypes_type))
print('Use the following columns types when reading the CSV file:')
print(column_types)

Use the following columns types when reading the CSV file:
{'tweet_id': 'uint64', 'user_id': 'uint64', 'tweet': 'object', 'key': 'category', 'key_group': 'category', 'retweets': 'uint32', 'favorites': 'uint32', 'mentions': 'category', 'hashtags': 'category', 'geo': 'float32', 'urls': 'object', 'permalink': 'object', 'username': 'object', 'key_mention': 'category', 'key_group_mention': 'category', 'lang': 'category', 'tweet_length': 'uint16', 'tweet_word_count': 'uint8', 'hashtags_count': 'uint8', 'mentions_count': 'uint8'}
